In [296]:
import nltk
import contractions, re
import pandas as pd
from string import punctuation
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [160]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/avanteesh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/avanteesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/avanteesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/avanteesh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Loading datasets
## data sources: kaggle

In [161]:
loaded_dataset = pd.read_csv("datasets/customer_support_tickets.csv")
loaded_dataset

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8464,8465,David Todd,adam28@example.net,22,Female,LG OLED,2021-12-08,Product inquiry,Installation support,My {product_purchased} is making strange noise...,Open,NaN,Low,Phone,NaN,NaN,NaN
8465,8466,Lori Davis,russell68@example.com,27,Female,Bose SoundLink Speaker,2020-02-22,Technical issue,Refund request,I'm having an issue with the {product_purchase...,Open,NaN,Critical,Email,NaN,NaN,NaN
8466,8467,Michelle Kelley,ashley83@example.org,57,Female,GoPro Action Camera,2021-08-17,Technical issue,Account access,I'm having an issue with the {product_purchase...,Closed,Eight account century nature kitchen.,High,Social media,2023-06-01 09:44:22,2023-06-01 04:31:22,3.0
8467,8468,Steven Rodriguez,fpowell@example.org,54,Male,PlayStation,2021-10-16,Product inquiry,Payment issue,I'm having an issue with the {product_purchase...,Closed,We seat culture plan.,Medium,Email,2023-06-01 18:28:24,2023-06-01 05:32:24,3.0


# Getting information about dataset

In [162]:
loaded_dataset.describe()

,Ticket ID,Customer Age,Customer Satisfaction Rating
count,8469.000000,8469.000000,2769.000000
mean,4235.000000,44.026804,2.991333
std,2444.934048,15.296112,1.407016
min,1.000000,18.000000,1.000000
25%,2118.000000,31.000000,2.000000
50%,4235.000000,44.000000,3.000000
75%,6352.000000,57.000000,4.000000
max,8469.000000,70.000000,5.000000


In [163]:
loaded_dataset.dtypes

Ticket ID                         int64
Customer Name                    object
Customer Email                   object
Customer Age                      int64
Customer Gender                  object
Product Purchased                object
Date of Purchase                 object
Ticket Type                      object
Ticket Subject                   object
Ticket Description               object
Ticket Status                    object
Resolution                       object
Ticket Priority                  object
Ticket Channel                   object
First Response Time              object
Time to Resolution               object
Customer Satisfaction Rating    float64
dtype: object

# Checking for Null (NaN) values!

In [164]:
loaded_dataset.isnull().sum()

Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Product Purchased                  0
Date of Purchase                   0
Ticket Type                        0
Ticket Subject                     0
Ticket Description                 0
Ticket Status                      0
Resolution                      5700
Ticket Priority                    0
Ticket Channel                     0
First Response Time             2819
Time to Resolution              5700
Customer Satisfaction Rating    5700
dtype: int64

In [325]:
label_encoder = LabelEncoder()
data = pd.DataFrame({
   "ticket": loaded_dataset['Ticket Description'],
    "label": loaded_dataset['Ticket Type'],
    "label_no": label_encoder.fit_transform(loaded_dataset['Ticket Type'])
})
data

,ticket,label,label_no
0,I'm having an issue with the {product_purchase...,Technical issue,4
1,I'm having an issue with the {product_purchase...,Technical issue,4
2,I'm facing a problem with my {product_purchase...,Technical issue,4
3,I'm having an issue with the {product_purchase...,Billing inquiry,0
4,I'm having an issue with the {product_purchase...,Billing inquiry,0
...,...,...,...
8464,My {product_purchased} is making strange noise...,Product inquiry,2
8465,I'm having an issue with the {product_purchase...,Technical issue,4
8466,I'm having an issue with the {product_purchase...,Technical issue,4
8467,I'm having an issue with the {product_purchase...,Product inquiry,2


# 📌 NLP Pipeline for Classical Machine Learning Sentiment Analysis (Flowchart)

```mermaid
flowchart TD
    A[Raw Text] --> B[Tokenization]
    B --> C[Lowercasing]
    C --> D[Punctuation Removal]
    D --> E[Stop-Word Removal]
    E --> G[Lemmatization]
    G --> H[Vectorization: CountVectorizer or TF-IDF]
    H --> I[Train/Test ML Model]
    I --> J[Sentiment Prediction]




In [326]:
STOP_WORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def word_processor(text: str):
    def get_wordnet_tag(tag: str):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        return wordnet.NOUN  # default
    text = contractions.fix(text)
    text = text.lower()
    text = re.sub(r"[_-]", " ", re.sub(r'{(.*?)}', r'\1', text))
    raw_tokens = word_tokenize(text)
    tokens = [token for token in raw_tokens if token not in punctuation]
    pos_tags = nltk.pos_tag(tokens)
    filtered_tokens = [(token, tag) for token, tag in pos_tags if token not in STOP_WORDS]
    lemma = [lemmatizer.lemmatize(word, get_wordnet_tag(tag)) for word, tag in filtered_tokens]
    return ' '.join(lemma)

In [328]:
# tokenization, lowercasing, punctuation and stop word removal and Lemmatization
data.ticket = data.ticket.apply(word_processor) 
data.ticket[0]

'issue product purchase please assist billing zip code 71701. appreciate request website address please double check email address try troubleshoot step mention user manual issue persist'

## Creating Training and Testing Sets

To evaluate the model effectively, the dataset is partitioned using an **80:20 split**:

- **Training Set (80%)** — used to fit and learn model parameters.
- **Test Set (20%)** — used to assess the model's performance on unseen data.


In [329]:
X_train, X_test, Y_train, Y_test = train_test_split(
  data.ticket,data.label_no, test_size=0.2,  # 20% samples will go into test
  random_state=52,stratify=data.label_no
)

In [314]:
print("X_train: ")
print(X_train)
print("\nY_train: ")
print(Y_train)

X_train: 
195     recently set product purchase fail connect ava...
1221    issue product purchase please assist 0 2 21 50...
5055    issue product purchase please assist 10.00 ite...
277     issue product purchase please assist find reas...
812     product purchase crashed lose data store way r...
                              ...                        
3199    face issue log product purchase account say ac...
4698    encounter data loss issue product purchase fil...
8049    face problem product purchase product purchase...
7095    product purchase make strange noise function p...
5539    issue product purchase please assist concern s...
Name: ticket, Length: 6775, dtype: object

Y_train: 
195     3
1221    2
5055    4
277     0
812     3
       ..
3199    2
4698    4
8049    3
7095    0
5539    4
Name: label_no, Length: 6775, dtype: int64


# Training Sample One
## Algorithm: KNN (K Nearest Neighbors)

In [336]:
from sklearn.neighbors import KNeighborsClassifier

knn_pipeline = Pipeline([
  ("tfidf_vectorizer", TfidfVectorizer()),
  ("knn_model", KNeighborsClassifier())
])
knn_pipeline.fit(X_train, Y_train)

,steps,"[('tfidf_vectorizer', ...), ('knn_model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [338]:
y_prediction = knn_pipeline.predict(X_test)
print(classification_report(Y_test, y_prediction))

              precision    recall  f1-score   support

           0       0.21      0.25      0.23       327
           1       0.22      0.27      0.24       339
           2       0.19      0.14      0.16       328
           3       0.23      0.25      0.24       351
           4       0.18      0.14      0.15       349

    accuracy                           0.21      1694
   macro avg       0.21      0.21      0.21      1694
weighted avg       0.21      0.21      0.21      1694

